# Tutorial 03: Análisis de Valores Faltantes

Este tutorial cubre el uso del módulo **null_analysis** de enahopy para:
- Analizar patrones de valores faltantes en datos ENAHO
- Detectar columnas problemáticas
- Generar reportes de calidad de datos
- Aplicar estrategias de imputación recomendadas

## ¿Por qué es importante?

Los valores faltantes son comunes en encuestas como ENAHO debido a:
- Preguntas condicionales (skip patterns)
- No respuestas
- Errores de captura
- Casos no aplicables

El análisis adecuado permite identificar si los nulos son:
- **MCAR** (Missing Completely At Random): Totalmente aleatorios
- **MAR** (Missing At Random): Relacionados con variables observadas
- **MNAR** (Missing Not At Random): Relacionados con valores no observados

In [ ]:
# Instalación (si es necesario)
# !pip install enahopy

import enahopy
from enahopy.loader import ENAHODataDownloader
from enahopy.loader.io import ENAHOLocalReader
from enahopy.null_analysis import ENAHONullAnalyzer
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print(f"enahopy versión: {enahopy.__version__}")

## 1. Cargar Datos con Valores Faltantes

### 1.1 Módulo 05 (Empleo e Ingresos)

Este módulo tiene muchos valores faltantes debido a preguntas condicionales:
- Ingresos solo aplican a trabajadores
- Ciertas preguntas son solo para dependientes/independientes
- Horas trabajadas solo para ocupados

In [6]:
year = 2022
import glob
import os

# Inicializar lector
dta_files_05 = glob.glob(f".enaho_cache/modulo_05_{year}/*.dta")
file_path_05 = dta_files_05[0]
reader_05 = ENAHOLocalReader(file_path=file_path_05, verbose=False)
df_modulo05, validation_05 = reader_05.read_data(columns=[
    'conglome', 'vivienda', 'hogar', 'codperso',  # Identificadores
    'ocu500',   # Condición de actividad
    'p506',      # Ocupación principal
    'p507',      # Categoría ocupacional
    'p511a',     # Horas trabajadas
    'i524a1',    # Ingreso trabajo dependiente
    'i530a',     # Ingreso trabajo independiente
    'p523',      # Tiene contrato escrito
    'p524a1'     # Tipo de contrato
            ])
print(f"\n✅ Datos cargados:")
print(f"   Filas: {len(df_modulo05):,}")
print(f"   Columnas: {len(df_modulo05.columns)}")


✅ Datos cargados:
   Filas: 87,661
   Columnas: 12


### 1.2 Exploración Inicial de Nulos

In [7]:
# Conteo básico de nulos
null_counts = df_modulo05.isnull().sum()
null_pcts = (null_counts / len(df_modulo05) * 100).sort_values(ascending=False)

print("📊 Top 15 columnas con más valores faltantes:")
print("=" * 60)
for col, pct in null_pcts.head(15).items():
    n_nulls = null_counts[col]
    print(f"{col:20s}: {n_nulls:6,} ({pct:5.1f}%)")

# Estadísticas generales
total_cells = df_modulo05.shape[0] * df_modulo05.shape[1]
total_nulls = df_modulo05.isnull().sum().sum()
null_rate = total_nulls / total_cells * 100

print(f"\n📈 Estadísticas generales:")
print(f"   Total de celdas: {total_cells:,}")
print(f"   Celdas con nulos: {total_nulls:,}")
print(f"   Tasa de nulos: {null_rate:.2f}%")

📊 Top 15 columnas con más valores faltantes:
p524a1              : 62,318 ( 71.1%)
p523                : 62,021 ( 70.8%)
i524a1              : 62,021 ( 70.8%)
i530a               : 60,199 ( 68.7%)
p511a               : 50,150 ( 57.2%)
p506                : 22,421 ( 25.6%)
p507                : 22,421 ( 25.6%)
conglome            :      0 (  0.0%)
vivienda            :      0 (  0.0%)
hogar               :      0 (  0.0%)
codperso            :      0 (  0.0%)
ocu500              :      0 (  0.0%)

📈 Estadísticas generales:
   Total de celdas: 1,051,932
   Celdas con nulos: 341,551
   Tasa de nulos: 32.47%


## 2. Análisis Avanzado con ENAHONullAnalyzer

### 2.1 Inicializar Analizador y Detectar Patrones

In [8]:
# Inicializar analizador
analyzer = ENAHONullAnalyzer(verbose=True)

# Analizar dataset
print("🔍 Analizando patrones de valores faltantes...\n")
result = analyzer.analyze(df_modulo05)

print(f"\n✅ Análisis completado:")
print(f"   Patrones detectados: {len(result['patterns']) if isinstance(result['patterns'], list) else 'N/A'}")
print(f"   Recomendaciones generadas: {len(result['recommendations'])}")

🔍 Analizando patrones de valores faltantes...


✅ Análisis completado:
   Patrones detectados: N/A
   Recomendaciones generadas: 9


### 2.2 Examinar Patrones Detectados

In [9]:
# Ver información sobre patrones
print("🔎 Información de patrones de valores faltantes:")
print("=" * 70)

patterns = result['patterns']
if isinstance(patterns, dict):
    print(f"\nTipo de patrones detectado: Diccionario")
    print(f"Claves disponibles: {list(patterns.keys())}")
    for key, value in patterns.items():
        print(f"\n  {key}: {value}")
elif isinstance(patterns, list) and len(patterns) > 0:
    for i, pattern in enumerate(patterns[:5], 1):
        print(f"\nPatrón {i}:")
        print(f"  Tipo: {pattern.get('type', 'N/A')}")
        print(f"  Columnas afectadas: {len(pattern.get('columns', []))}")
        print(f"  Severidad: {pattern.get('severity', 'N/A')}")
        if 'description' in pattern:
            print(f"  Descripción: {pattern['description']}")
else:
    print("\nNo se detectaron patrones específicos o el análisis está en progreso.")

🔎 Información de patrones de valores faltantes:

Tipo de patrones detectado: Diccionario
Claves disponibles: ['global', 'numeric', 'categorical']

  global: PatternResult(pattern_type=<MissingDataPattern.MNAR: 'Missing Not At Random'>, severity=<PatternSeverity.HIGH: 'high'>, affected_columns=['p506', 'p507', 'p511a', 'p523', 'p524a1', 'i530a', 'i524a1'], percentage_missing=32.46892384678858, confidence=0.8, details={'total_values': 1051932, 'null_values': 341551, 'overall_percentage': 32.46892384678858, 'columns_with_nulls': 7, 'rows_with_nulls': 87661, 'complete_rows': 0, 'complete_columns': 5, 'null_correlation_mean': nan}, recommendations=['Los datos no faltan aleatoriamente', 'Investigue las causas de los valores faltantes', 'Porcentaje significativo de datos faltantes'])

  numeric: PatternResult(pattern_type=<MissingDataPattern.MNAR: 'Missing Not At Random'>, severity=<PatternSeverity.CRITICAL: 'critical'>, affected_columns=['i530a', 'i524a1'], percentage_missing=69.711730416034

### 2.3 Revisar Recomendaciones

In [ ]:
# Ver recomendaciones
print("💡 Recomendaciones para manejo de valores faltantes:")
print("=" * 70)

recommendations = result['recommendations']
if len(recommendations) > 0:
    for i, rec in enumerate(recommendations[:10], 1):
        # Las recomendaciones pueden ser strings o dicts
        if isinstance(rec, str):
            print(f"\n{i}. {rec}")
        elif isinstance(rec, dict):
            print(f"\n{i}. {rec.get('action', 'N/A')}")
            if 'columns' in rec:
                print(f"   Columnas: {', '.join(rec['columns'][:3])}..." if len(rec['columns']) > 3 else f"   Columnas: {', '.join(rec['columns'])}")
            if 'reason' in rec:
                print(f"   Razón: {rec['reason']}")
        else:
            print(f"\n{i}. {rec}")
else:
    print("\nNo se generaron recomendaciones específicas.")
    print("Puede usar las estrategias generales según el % de valores faltantes.")

## 3. Generar Reportes Detallados

### 3.1 Resumen del Análisis

In [ ]:
# Mostrar resumen del análisis
print("📋 Resumen del Análisis de Valores Faltantes")
print("=" * 70)

# Resumen general
if 'summary' in result:
    print("\n📊 Resumen General:")
    summary = result['summary']
    if isinstance(summary, dict):
        for key, value in summary.items():
            if key not in ['columns_analysis', 'row_analysis', 'patterns']:
                print(f"   {key}: {value}")

### 3.2 Análisis de Correlación de Nulos

Identificar si los nulos en ciertas columnas están correlacionados:

In [ ]:
# Seleccionar columnas con >10% de nulos para análisis de correlación
high_null_cols = null_pcts[null_pcts > 10].index[:10].tolist()

if len(high_null_cols) >= 2:
    # Crear matriz de indicadores de nulos
    null_matrix = df_modulo05[high_null_cols].isnull().astype(int)
    
    # Calcular correlación entre patrones de nulos
    null_corr = null_matrix.corr()
    
    print("🔗 Correlación entre patrones de valores faltantes:")
    print("   (Valores altos indican que los nulos ocurren juntos)\n")
    
    # Encontrar pares con alta correlación (>0.7)
    high_corr_pairs = []
    for i in range(len(null_corr.columns)):
        for j in range(i+1, len(null_corr.columns)):
            corr_val = null_corr.iloc[i, j]
            if corr_val > 0.7:
                high_corr_pairs.append((
                    null_corr.columns[i],
                    null_corr.columns[j],
                    corr_val
                ))
    
    if high_corr_pairs:
        print(f"   Pares con correlación >0.7: {len(high_corr_pairs)}\n")
        for col1, col2, corr in sorted(high_corr_pairs, key=lambda x: x[2], reverse=True)[:5]:
            print(f"   {col1} ↔ {col2}: {corr:.3f}")
    else:
        print("   No se encontraron pares con correlación >0.7")

## 4. Análisis por Subgrupos

### 4.1 Analizar Nulos por Condición de Actividad (ocu500)

In [ ]:
# Verificar si existe la columna ocu500 (condición de actividad)
if 'ocu500' in df_modulo05.columns:
    print("📊 Análisis de nulos por Condición de Actividad (ocu500):\n")
    
    # Agrupar por condición de actividad
    for category in df_modulo05['ocu500'].dropna().unique()[:3]:
        subset = df_modulo05[df_modulo05['ocu500'] == category]
        null_rate = subset.isnull().sum().sum() / (subset.shape[0] * subset.shape[1]) * 100
        
        print(f"   Categoría {category}: {null_rate:.2f}% nulos (n={len(subset):,})")
else:
    print("⚠️ Columna 'ocu500' no encontrada en el dataset")

## 5. Estrategias de Imputación

### 5.1 Identificar Columnas Candidatas para Imputación

In [ ]:
# Categorizar columnas según porcentaje de nulos
low_null_cols = null_pcts[(null_pcts > 0) & (null_pcts < 5)].index.tolist()
medium_null_cols = null_pcts[(null_pcts >= 5) & (null_pcts < 30)].index.tolist()
high_null_cols = null_pcts[null_pcts >= 30].index.tolist()

print("🎯 Clasificación de columnas según % de nulos:\n")
print(f"   Bajo (<5%):     {len(low_null_cols)} columnas")
print(f"   Medio (5-30%):  {len(medium_null_cols)} columnas")
print(f"   Alto (>30%):    {len(high_null_cols)} columnas")

print("\n💡 Recomendaciones de estrategia:")
print("\n   Bajo (<5%):")
print("   - Imputación por media/mediana (numéricos)")
print("   - Imputación por moda (categóricos)")
print("   - KNN imputation")

print("\n   Medio (5-30%):")
print("   - Análisis de patrones antes de imputar")
print("   - Multiple imputation (MICE)")
print("   - Modelos predictivos")

print("\n   Alto (>30%):")
print("   - Considerar eliminar columna")
print("   - Crear indicador de nulo")
print("   - Analizar si son nulos estructurales (skip patterns)")

### 5.2 Ejemplo: Imputación Simple en Columnas de Bajo % de Nulos

In [ ]:
# Seleccionar columnas numéricas con <5% de nulos
numeric_cols = df_modulo05.select_dtypes(include=[np.number]).columns
low_null_numeric = [col for col in low_null_cols if col in numeric_cols]

if low_null_numeric:
    # Tomar primera columna como ejemplo
    example_col = low_null_numeric[0]
    
    print(f"📝 Ejemplo de imputación: {example_col}\n")
    print(f"   Valores faltantes: {df_modulo05[example_col].isnull().sum()} ({null_pcts[example_col]:.2f}%)")
    print(f"   Media: {df_modulo05[example_col].mean():.2f}")
    print(f"   Mediana: {df_modulo05[example_col].median():.2f}")
    
    # Crear copia e imputar
    df_imputed = df_modulo05.copy()
    df_imputed[example_col] = df_imputed[example_col].fillna(df_imputed[example_col].median())
    
    print(f"\n   ✅ Imputación completada con mediana")
    print(f"   Valores faltantes después: {df_imputed[example_col].isnull().sum()}")
else:
    print("No hay columnas numéricas con <5% de nulos para ejemplo")

## 6. Análisis de Datos Completos

### 6.1 Filas Completamente Sin Nulos

In [ ]:
# Identificar filas sin ningún valor nulo
complete_rows = df_modulo05.dropna(how='any')
complete_pct = len(complete_rows) / len(df_modulo05) * 100

print(f"📊 Análisis de datos completos:\n")
print(f"   Total de filas: {len(df_modulo05):,}")
print(f"   Filas completas (sin nulos): {len(complete_rows):,}")
print(f"   Porcentaje de filas completas: {complete_pct:.2f}%")

# ¿Cuántas columnas necesitaríamos eliminar para tener 50% de filas completas?
cols_sorted_by_nulls = null_pcts.sort_values(ascending=False)
df_temp = df_modulo05.copy()

for i, col in enumerate(cols_sorted_by_nulls.index, 1):
    if col in df_temp.columns:
        df_temp = df_temp.drop(columns=[col])
        complete_temp = df_temp.dropna(how='any')
        pct_complete = len(complete_temp) / len(df_temp) * 100
        
        if pct_complete >= 50:
            print(f"\n   💡 Para lograr 50% de filas completas:")
            print(f"      Eliminar {i} columnas con más nulos")
            print(f"      Resultaría en {len(complete_temp):,} filas completas ({pct_complete:.1f}%)")
            break

## 7. Mejores Prácticas y Recomendaciones

### Resumen de Aprendizajes

In [ ]:
print("="*70)
print("💡 MEJORES PRÁCTICAS PARA MANEJO DE VALORES FALTANTES")
print("="*70)

print("""
1. SIEMPRE ANALIZAR ANTES DE IMPUTAR
   ✓ Usar ENAHONullAnalyzer para entender patrones
   ✓ Identificar si son nulos estructurales (skip patterns)
   ✓ Verificar correlaciones entre columnas con nulos

2. ESTRATEGIAS SEGÚN % DE NULOS
   ✓ <5%: Imputación simple (media/mediana/moda)
   ✓ 5-30%: Análisis detallado + MICE o modelos
   ✓ >30%: Considerar eliminar o crear indicador

3. NULOS ESTRUCTURALES EN ENAHO
   ✓ Preguntas condicionales son la causa principal
   ✓ Ej: Ingresos solo para trabajadores (ocu500)
   ✓ NO imputar estos nulos - son legítimos

4. DOCUMENTAR DECISIONES
   ✓ Registrar qué columnas se imputaron
   ✓ Registrar método utilizado
   ✓ Comparar resultados antes/después

5. VALIDAR DESPUÉS DE IMPUTAR
   ✓ Verificar distribuciones
   ✓ Comprobar rangos válidos
   ✓ Análisis de sensibilidad
""")

print("="*70)

## Conclusiones

En este tutorial aprendiste a:

1. ✅ Cargar datos con valores faltantes usando `ENAHOLocalReader`
2. ✅ Realizar análisis exploratorio de nulos
3. ✅ Usar `ENAHONullAnalyzer` para detectar patrones automáticamente
4. ✅ Interpretar resultados del análisis (diccionarios con patterns y recommendations)
5. ✅ Analizar correlaciones entre patrones de nulos
6. ✅ Identificar estrategias de imputación apropiadas
7. ✅ Comparar calidad de datos entre subgrupos
8. ✅ Aplicar mejores prácticas para manejo de valores faltantes

### Próximos Pasos

- **Tutorial 04**: Pipeline completo integrando `loader` + `merger` + `null_analysis`
- Aplicar estas técnicas en análisis reales de pobreza e inequidad
- Experimentar con diferentes métodos de imputación